### Курсовой проект для курса "Python для Data Science"

Выполнил: Алексей Некрасов

Материалы к проекту (файлы):
train.csv
test.csv

Задание:
Используя данные из обучающего датасета (train.csv), построить модель для предсказания цен на недвижимость (квартиры).
С помощью полученной модели, предсказать цены для квартир из тестового датасета (test.csv).

Целевая переменная:
Price

Метрика качества:
R2 - коэффициент детерминации (sklearn.metrics.r2_score)

Требования к решению: R2 > 0.6

### 1. Работает с тренировочным датасетом

In [1]:
# Загрузка необходимых библиотек

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings('ignore')


# Загрузка тренировочных данных из файла

TRAIN_DATASET_PATH='C:/GoogleDisk/Учение/GB_Python-Data Science-1/project_task/project_task/train.csv'
TEST_DATASET_PATH='C:/GoogleDisk/Учение/GB_Python-Data Science-1/project_task/project_task/test.csv'

data_train = pd.read_csv(TRAIN_DATASET_PATH, index_col="Id")
# data_train.info()

In [2]:
# Обрабатка пропущенных значений

LifeSquare_mean = data_train["LifeSquare"].mean()
print(LifeSquare_mean)
data_train["LifeSquare"].fillna(LifeSquare_mean, inplace=True)

Healthcare_1_mean = data_train["Healthcare_1"].mean()
print(Healthcare_1_mean)
data_train["Healthcare_1"].fillna(Healthcare_1_mean, inplace=True)

37.19964482767308
1142.9044598231449


In [3]:
# Обрабатка категориальных признаков

data_train = pd.concat([data_train,pd.get_dummies(data_train['Ecology_2'])],axis=1)
data_train.drop("Ecology_2", axis=1, inplace=True)

data_train = pd.concat([data_train,pd.get_dummies(data_train['Ecology_3'])],axis=1)
data_train.drop("Ecology_3", axis=1, inplace=True)

data_train = pd.concat([data_train,pd.get_dummies(data_train['Shops_2'])],axis=1)
data_train.drop("Shops_2", axis=1, inplace=True)

In [4]:
# Создаение тренировачных датафреймов

X = pd.DataFrame(data_train)
y = pd.DataFrame(data_train["Price"])

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25,random_state=42)

# Удаление целевого показателя из таблиц признаков
X_train.drop("Price", axis=1, inplace=True)
X_valid.drop("Price", axis=1, inplace=True)

In [5]:
# Стандартизация признаков

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)

X_valid_scaled = scaler.fit_transform(X_valid)
X_valid_scaled = pd.DataFrame(X_valid_scaled, columns=X_valid.columns)



In [6]:
# Поиск наилучших параметров (для ускорения выполнения скрипта)

# %%time
# # Опредение нилучших параметров для модели

# parameters = {
#     'n_estimators': [150],
#     'max_features': (5,7,9),
#     'max_depth': [17],
# }
# grid = GridSearchCV(
#     estimator=RandomForestRegressor(random_state=42),
#     param_grid=parameters,
#     scoring='r2',
# )

# grid.fit(X_train, y_train)

# print('Лучшие параметры: ',grid.best_params_)

# y_pred = grid.predict(X_train)

# print('Коэф. детерминации на тренировочных данных: ',sklearn.metrics.r2_score(y_train,y_pred))

# y_pred2 = grid.predict(X_valid)

# print('Коэф. детерминации на тренировочных данных: ',sklearn.metrics.r2_score(y_valid,y_pred2))

In [7]:
%%time
# Создание модели, тренировка, предсказание.

rfr = RandomForestRegressor(n_estimators=150,max_features=5, max_depth=17 ,random_state=42)

rfr.fit(X_train, y_train)

y_pred = rfr.predict(X_train)

print('Коэф. детерминации на тренировочных данных: ',sklearn.metrics.r2_score(y_train,y_pred))

y_pred2 = rfr.predict(X_valid)

print('Коэф. детерминации на тренировочных данных: ',sklearn.metrics.r2_score(y_valid,y_pred2))

Коэф. детерминации на тренировочных данных:  0.9500033330382888
Коэф. детерминации на тренировочных данных:  0.7327435630738615
Wall time: 5.6 s


### 2. Работаем с тестовым датасетом

In [8]:
# Загрузка тестовых данных из файла

data_test = pd.read_csv(TEST_DATASET_PATH, index_col="Id")
# data_test.head()

In [9]:
# Обработка пропущенных значений

LifeSquare_mean = data_test["LifeSquare"].mean()
print(LifeSquare_mean)
data_test["LifeSquare"].fillna(LifeSquare_mean, inplace=True)

Healthcare_1_mean = data_test["Healthcare_1"].mean()
print(Healthcare_1_mean)
data_test["Healthcare_1"].fillna(Healthcare_1_mean, inplace=True)

36.15881045508672
1146.6572626763248


In [10]:
# Обработка категориальных признаков

data_test = pd.concat([data_test,pd.get_dummies(data_test['Ecology_2'])],axis=1)
data_test.drop("Ecology_2", axis=1, inplace=True)

data_test = pd.concat([data_test,pd.get_dummies(data_test['Ecology_3'])],axis=1)
data_test.drop("Ecology_3", axis=1, inplace=True)

data_test = pd.concat([data_test,pd.get_dummies(data_test['Shops_2'])],axis=1)
data_test.drop("Shops_2", axis=1, inplace=True)

In [11]:
# Создание датафрейма для предсказания и Стандартизация признаков.

X_test = pd.DataFrame(data_test)

X_test_scaled = scaler.fit_transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [12]:
# Предсказание.

y_test = rfr.predict(X_test)

In [13]:
# Запись индексов наблюдений в отдельную колонку.

data_test.reset_index(drop=False, inplace=True)

In [14]:
# Создание датафрейма для вывода результатов предсказания.

test = pd.DataFrame({'id':data_test['Id'],'Price':y_test})
# test

In [15]:
# Выгрузка резульатов предсказания в csv.

test[['id','Price']].to_csv('sampleSubmissionNek4.csv',index=False)

#### Результат
Предсказание на тестовых данных загружено в соревнование на Kaggle.com.

Полученный коэфициент детерминации: 0.73658.

Соревнование: https://www.kaggle.com/c/realestatepriceprediction/leaderboard

Мой профиль на Kaggle.com: https://www.kaggle.com/alekseinekrasov